# Generate tree

In [7]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import os
import json

import random
import time as time
import pandas as pd
import numpy as np
from pathlib import Path

from rmgpy.molecule.group import Group
from rmgpy.data.base import Entry
from rmgpy.data.thermo import ThermoData

from tree.thermo import ThermoGroups, average_thermo_data
from tree.utils import make_mol
from tree.parameters import Ts

# Get data

In [8]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")
hbi_unc_df

,radical_smiles,radical_resonance_smiles,closed_shell_smiles,closed_shell_H298 (kcal/mol),closed_shell_S298 (cal/mol/K),closed_shell_Cp300 (cal/mol/K),closed_shell_Cp400 (cal/mol/K),closed_shell_Cp500 (cal/mol/K),closed_shell_Cp600 (cal/mol/K),closed_shell_Cp800 (cal/mol/K),...,unc_HBI_H298 (kcal/mol),unc_HBI_S298 (cal/mol/K),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
0,[O]C(=O)OC(O)(O)O,[O]C(=O)OC(O)(O)O,O=C(O)OC(O)(O)O,-272.617591,98.406448,31.562314,36.827897,41.095602,44.234417,48.821415,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,[O]C(O)(O)OC(=O)O,[O]C(O)(O)OC(=O)O,O=C(O)OC(O)(O)O,-272.617591,98.406448,31.562314,36.827897,41.095602,44.234417,48.821415,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,O=C(O)O[C](O)O,O=C(O)O[C](O)O,O=C(O)OC(O)O,-231.363843,94.127419,27.214474,31.965583,36.032333,39.309560,44.059044,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)OO,-234.219312,145.598548,52.706616,62.438776,71.442505,78.070363,87.443365,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,O=[C]OC(O)(O)O,O=[C]OC(O)(O)O,O=COC(O)(O)O,-216.499044,92.684651,26.935163,30.962696,34.637667,37.592447,42.375430,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,C1=C[CH]C=1,[C]1=CC=C1,C1=CC=C1,104.333500,57.818020,15.504000,19.920000,23.474000,26.261000,30.329000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2985,C=C=C1C=[C]C1,C=C=C1C=[C]C1,C=C=C1C=CC1,91.366874,50.394958,22.416919,29.489266,35.000461,39.185145,45.162263,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2986,[CH2]C1=CC#CC1,[CH2]C1=CC#CC1,CC1=CC#CC1,131.227000,72.956831,23.409000,30.017000,34.619000,39.828000,46.529000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2987,[CH2]C1=CC#CC1,C=C1[CH]C#CC1,C=C1CC#CC1,136.767000,72.435147,22.819000,29.017000,34.189000,38.448000,45.419000,...,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [9]:
with open("../data/splits/random.json", "r") as f:
    train_inds, test_inds = json.load(f)

In [10]:
train_df = hbi_unc_df.loc[train_inds, :]

In [11]:
mols = train_df["radical_resonance_smiles"].apply(make_mol)
mols = mols.to_list()

In [12]:
HBI_corrections = train_df.apply(lambda x: ThermoData(
    H298=(x["HBI_H298 (kcal/mol)"], "kcal/mol", "+|-", x["unc_HBI_H298 (kcal/mol)"]),
    S298=(x["HBI_Sint298 (cal/mol/K)"], "cal/mol/K", "+|-", x["unc_HBI_Sint298 (cal/mol/K)"]),
    Cpdata=([x[f"HBI_Cp{T} (cal/mol/K)"] for T in Ts], "cal/mol/K", "+|-", [x[f"unc_HBI_Cp{T} (cal/mol/K)"] for T in Ts]),
    Tdata=(Ts, "K"),
    comment=f"Radical thermo from {x['radical_source']} and closed shell thermo from {x['closed_shell_thermo_source']}"
), axis=1)
HBI_corrections = HBI_corrections.to_list()

In [13]:
mols_corrections_all = list(zip(mols, HBI_corrections))

# Generate tree

In [16]:
n_jobs = 4
# save_name = "tree-base_case-random_split"
save_dir = "test"
os.makedirs(f"../models/{save_dir}/", exist_ok=True)
mols_corrections = mols_corrections_all

In [ ]:
#clean tree
tree = ThermoGroups(
    label="radical",
    name="HBI correction",
)

tree.entries["Root"] = Entry(
    index = 0,
    label = "Root",
    item = Group().from_adjacency_list(f"""1 * R u[1,2,3,4]"""),
    data = None,
    data_count = 0,
    parent = None,
)
tree.entries["RJ1"] = Entry(
    index = 0,
    label = "RJ1",
    item = Group().from_adjacency_list(f"""1 * R u1"""),
    data = None,
    data_count = 0,
    parent = tree.entries["Root"],
)

tree.entries["Root"].children = [tree.entries["RJ1"]]
tree.top = [tree.entries["Root"]]


start = time.time()
template_mol_map_exact = tree.generate_tree(mols_corrections=mols_corrections, obj=None, Ts=None, nprocs=1, min_splitable_entry_num=2,
                                          min_mols_corrections_to_spawn=20, max_batch_size=np.inf, outlier_fraction=0.02, stratum_num=8,
                                          new_fraction_threshold_to_reopt_node=0.25, extension_iteration_max=2, extension_iteration_item_cap=100, 
                                            min_mols_corrections_to_split=1, n_jobs=n_jobs)
end = time.time()
print("Tree generation:")
print(end-start)

tree.check_tree()

start = time.time()
template_mol_map = tree.get_molecule_matches(mols_corrections=mols_corrections,
                                                     exact_matches_only=False, n_jobs=n_jobs)
end = time.time()
print("Mol mapping:")
print(end-start)

tree.regularize(template_mol_map)

start = time.time()
tree.make_corrections_from_template_mol_map(template_mol_map, n_jobs=n_jobs)
end = time.time()
print("Make corrections:")
print(end-start)

tree.check_tree()

# def add_children(old_node, new_node):
#     for old_child in old_node.children:
#         tree.entries[old_child.label] = deepcopy(old_child)
#         tree.entries[old_child.label].parent = new_node
#         new_node.children.append(tree.entries[old_child.label])
#         add_children(old_child, tree.entries[old_child.label])
# tree.entries["RJ2_triplet"] = deepcopy(thermo_database.groups["radical"].entries["RJ2_triplet"])
# tree.entries["RJ2_triplet"].parent = tree.entries["Root"]
# tree.entries["Root"].children.append(tree.entries["RJ2_triplet"])
# add_children(thermo_database.groups["radical"].entries["RJ2_triplet"], tree.entries["RJ2_triplet"])

# tree.entries["RJ3"] = deepcopy(thermo_database.groups["radical"].entries["RJ3"])
# tree.entries["RJ3"].parent = tree.entries["Root"]
# tree.entries["Root"].children.append(tree.entries["RJ3"])
# add_children(thermo_database.groups["radical"].entries["RJ3"], tree.entries["RJ3"])

tree.save(f"../models/{save_dir}/tree.py")

INFO:root:Getting molecule matches...
[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 632 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 2072 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 2711 out of 2711 | elapsed:    6.7s finished
INFO:root:2711.0
INFO:root:2
INFO:root:4
INFO:root:6
INFO:root:8
INFO:root:10
INFO:root:12
INFO:root:13
INFO:root:14
INFO:root:16
INFO:root:17
INFO:root:18
INFO:root:20
INFO:root:21
INFO:root:22
INFO:root:24
INFO:root:25
INFO:root:27
INFO:root:29
INFO:root:31
INFO:root:33
INFO:root:35
INFO:root:37
INFO:root:39
INFO:root:41
INFO:root:42
INFO:root:43
INFO:root:45
INFO:root:47
INFO:root:49
INFO:root:51
INFO:root:53
INFO:root:55
INFO:root:57
INFO:root:59
INFO:root:61
INFO:root:63
INFO:root:64
INFO:root:66
INFO:root:67
INFO:root:iteration_max achieved terminating early
INFO:root:68
INFO:root:iteration_max achieved ter